In [1]:
import pandas as pd
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import joblib
import math

In [2]:
df = pd.read_csv('data/data_substituicao/labeled.csv')
print(len(df))
# train = df[:math.floor(0.7*len(df))]
# test  = df[math.floor(0.7*len(df)):]
# print(len(df), len(train), len(test))
# df = []

180


In [3]:
def extract_features(sentence):
  sentence_features = []
  for j in range(len(sentence)):
    word_feat = {
            'word': sentence[j].lower(),
            'capital_letter': sentence[j][0].isupper(),
            'all_capital': sentence[j].isupper(),
            'isdigit': sentence[j].isdigit(),
            'word_before': sentence[j].lower() if j==0 else sentence[j-1].lower(),
            'word_after:': sentence[j].lower() if j+1>=len(sentence) else sentence[j+1].lower(),
            'BOS': j==0,
            'EOS': j==len(sentence)-1
    }
    sentence_features.append(word_feat)
  return sentence_features

def separate_cols(arq):
    x = []
    y = []
    for i in range(len(arq)):
        x.append(extract_features(arq['text'][i].split()))
        y.append(arq['labels'][i].split())
    return x, y

x, y = separate_cols(df)
train_x = x[:math.floor(0.8*len(df))]
train_y = y[:math.floor(0.8*len(df))]
test_x = x[math.floor(0.8*len(df)):]
test_y = y[math.floor(0.8*len(df)):]

len(df), len(train_x), len(train_y), len(test_x), len(test_y)

(180, 144, 144, 36, 36)

In [98]:
model = sklearn_crfsuite.CRF(
    algorithm = 'l2sgd', 
    c2=0.4,
    max_iterations=100, 
    all_possible_transitions=True,
    verbose=False
)

model.fit(train_x, train_y)
y_pred = model.predict(test_x)

In [99]:
labels = list(model.classes_)
labels.remove('O')

f1 = metrics.flat_f1_score(test_y, y_pred, 
                      average='weighted', labels=labels)
print(f1)

0.732224905292508


In [100]:
print(metrics.flat_classification_report(
    test_y, y_pred, labels=labels, digits=3
))

                                             precision    recall  f1-score   support

                 B-nome_servidor_substituto      1.000     0.971     0.986        35
                 I-nome_servidor_substituto      0.938     0.972     0.955       109
                     B-matricula_substituto      0.493     0.972     0.654        36
                     I-matricula_substituto      0.480     0.973     0.643        37
                B-nome_servidor_substituido      0.974     1.000     0.987        37
                I-nome_servidor_substituido      0.962     1.000     0.980       100
                    B-matricula_substituido      0.000     0.000     0.000        36
                    I-matricula_substituido      0.000     0.000     0.000        39
  B-simbolo_do_cargo_objeto_da_substituicao      0.717     1.000     0.835        33
  I-simbolo_do_cargo_objeto_da_substituicao      0.708     0.944     0.810        36
                       B-inicio_da_vigencia      0.868     1.000

In [101]:
labels.remove('B-matricula_siape')
labels.remove('I-matricula_siape')
labels.remove('B-cargo_comissionado_objeto_da_substituicao')
labels.remove('I-cargo_comissionado_objeto_da_substituicao')
print(metrics.flat_classification_report(
    test_y, y_pred, labels=labels, digits=3
))

                                           precision    recall  f1-score   support

               B-nome_servidor_substituto      1.000     0.971     0.986        35
               I-nome_servidor_substituto      0.938     0.972     0.955       109
                   B-matricula_substituto      0.493     0.972     0.654        36
                   I-matricula_substituto      0.480     0.973     0.643        37
              B-nome_servidor_substituido      0.974     1.000     0.987        37
              I-nome_servidor_substituido      0.962     1.000     0.980       100
                  B-matricula_substituido      0.000     0.000     0.000        36
                  I-matricula_substituido      0.000     0.000     0.000        39
B-simbolo_do_cargo_objeto_da_substituicao      0.717     1.000     0.835        33
I-simbolo_do_cargo_objeto_da_substituicao      0.708     0.944     0.810        36
                     B-inicio_da_vigencia      0.868     1.000     0.930        33
   

In [97]:
# Save trained model
joblib.dump(model, "substituicao_ner.pkl")

['substituicao_ner.pkl']